# 3. Asynchronous Memory Copy

SGEMM 예제에서 측정한 수행시간을 보면 CPU / GPU 수행시간이 거의 비슷하다는 것을 볼 수 있습니다. 단순히 수행시간을 측정했고 속도가 빨라졌으니 좋은 일이지만, 따지고 보면 SGEMM 연산을 하는 동안 CPU는 아무것도 한 것이 없습니다. 즉, CUDA 연산이 끝날때까지 CPU는 CUDA의 동작이 끝나기를 기다린 것입니다. 이는 Memory를 복사하는 명령인 cudaMemcpy에서 CPU/GPU간 데이터 전송이 모두 끝나기 까지 다음 line으로 넘어가지 않기 때문입니다.

In [1]:
%%file sgemm_async_copy.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

typedef enum TARGET {HOST, DEVICE} TARGET;
typedef enum MEMTYPE {NORMAL, PINNED} MEMTYPE;

typedef struct {
    int width;
    int height;
    float *elements;
} Matrix;

__global__ void sgemm(Matrix A, Matrix B, Matrix C, 
                      const float alpha, const float beta, 
                      const int width, const int height) {
    int idx_x = blockDim.x * blockIdx.x + threadIdx.x;
    int idx_y = blockDim.y * blockIdx.y + threadIdx.y;
    int idx = idx_y * width + idx_x;
    
    if (idx_x >= width || idx_y >= height)
        return;
    
    float value = 0.f;
    for (int e = 0; e < width; e++)
        value = alpha * A.elements[idx_y * width + e] * B.elements[e * width + idx_x];
    C.elements[idx] = value + beta * C.elements[idx];
}

void InitMatrix(Matrix &mat, const int width, const int height, TARGET target = HOST, MEMTYPE memtype = NORMAL);

int main(int argv, char* argc[]) {
    Matrix A, B, C;
    Matrix dA, dB, dC;
    const float alpha = 2.f;
    const float beta = .5f;
    const int width = 2048;
    const int height = 2048;
    float elapsed_gpu;
    double elapsed_cpu;
    
    // Select Host memory type (NORMAL, PINNED)
    MEMTYPE memtype = PINNED;
    
    // CUDA Event Create to estimate elased time
    cudaEvent_t start, stop;
    struct timespec begin, finish;
    
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Initialize host matrix
    InitMatrix(A, width, height, HOST, memtype);
    InitMatrix(B, width, height, HOST, memtype);
    InitMatrix(C, width, height, HOST, memtype);

    // CUDA Memory Initialize
    InitMatrix(dA, width, height, DEVICE);
    InitMatrix(dB, width, height, DEVICE);
    InitMatrix(dC, width, height, DEVICE);
    
    // CUDA Operation
    cudaEventRecord(start, 0);
    clock_gettime(CLOCK_MONOTONIC, &begin);
    
    // Copy host data to the device (CUDA global memory)
    // TODO: Write Asynchronous CUDA Memcpy API (gpu -> cpu)
    
    //////////////
    
    // Launch GPU Kernel
    dim3 blockDim(16, 16);
    dim3 gridDim((width + blockDim.x - 1) / blockDim.x, (height + blockDim.y - 1) / blockDim.y);
    sgemm<<<gridDim, blockDim>>>(dA, dB, dC, alpha, beta, width, height);
    
    // Copy computation result from the Device the host memory
    // TODO: Write Asynchronous CUDA Memcpy API (cpu -> gpu)
    
    //////////////
    clock_gettime(CLOCK_MONOTONIC, &finish);
    cudaEventRecord(stop, 0);
    
    // Estimate CUDA operation time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    
    cudaEventElapsedTime(&elapsed_gpu, start, stop);
    printf("SGEMM CUDA Elapsed time: %f ms\n", elapsed_gpu);
    elapsed_cpu = (finish.tv_sec - begin.tv_sec);
    elapsed_cpu += (finish.tv_nsec - begin.tv_nsec) / 1000000000.0;
    printf("Host time: %f ms\n", elapsed_cpu * 1000);
    
    // finalize CUDA event
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    // Finalize
    cudaFree(dA.elements);
    cudaFree(dB.elements);
    cudaFree(dC.elements);
    
    if (memtype == NORMAL) {
        free(A.elements);
        free(B.elements);
        free(C.elements);
    }
    else {
        // TODO: Write pinned memory free API

        /////////////
    }
    
    return 0;
}

void InitMatrix(Matrix &mat, const int width, const int height, TARGET target, MEMTYPE memtype) {
    mat.width = width;
    mat.height = height;
    
    if (target == DEVICE) {
        cudaMalloc((void**)&mat.elements, width * height * sizeof(float));
    }
    else {
        if (memtype == NORMAL)
            mat.elements = (float*)malloc(width * height * sizeof(float));
        else
            // TODO: Write pinned memory allocation API
            
            /////////////
    
        for (int row = 0; row < height; row++) {
            for (int col = 0; col < width; col++) {
                mat.elements[row * width + col] = row * width + col * 0.001;
            }
        }
    }
}


Overwriting sgemm_async_copy.cu


In [2]:
! make sgemm_async_copy

nvcc sgemm_async_copy.cu --ptxas-options=--verbose -gencode=arch=compute_35,code=sm_35 -I/usr/local/cuda/samples/common/inc -o sgemm_async_copy
ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z5sgemm6MatrixS_S_ffii' for 'sm_35'
ptxas info    : Function properties for _Z5sgemm6MatrixS_S_ffii
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 13 registers, 384 bytes cmem[0]


In [3]:
! ./sgemm_async_copy

SGEMM CUDA Elapsed time: 30.752159 ms
Host time: 0.038772 ms


1. 처음 수행시간을 보면 비동기 동작은 확실히 일어나지 않았고, 처음과 같은 속도로 동작했습니다. 이는 비동기 동작을 하기 위해서는 Pinned Memory를 사용해야하기 때문입니다. 위 코드에서 malloc을 사용하던 코드에서 pinned memory를 사용할 수 있도록 수정해서 동작시간이 어떻게 변하는지 살펴보세요.

2. GPU 동작시간은 비동기 동작을 했을 떄와 동일하게 바뀌었습니다. 한편 Host의 수행시간이 엄청나게 줄었습니다. 이는 곧 Host단에서 GPU 메모리 복사를 기다리지 않는 비동기 동작이 활성화 되었기 때문입니다. 하지만 이렇게 하는 경우에는 연산결과를 제대로 반영하지 않은채 함수나 프로그램이 종료될 수 있기 때문에 동기화를 하게 됩니다. CUDA는 다양한 방법을 제공하는데 우선은 GPU에 대하여 동기화를 하겠습니다. CUDA에서 GPU 동작에 대하여 Host의 동작을 동기화하는 명령은 아래와 같습니다.

** cudaDeviceSynchronize() **

위 코드에서 *cudaDeviceSynchronize()*의 위치가 Host 수행시간을 확인하는 코드와의 순서에 따라서 다르게 나타날 수 있으니 주의하시기 바랍니다.